In [24]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import types, functions as F

In [25]:
spark = SparkSession.builder\
          .master("local[*]")\
          .appName("test")\
          .getOrCreate()

In [26]:
schema = types.StructType([
  types.StructField('hvfhs_license_num', types.StringType(), True),
  types.StructField('dispatching_base_num', types.StringType(), True),
  types.StructField('originating_base_num', types.StringType(), True),
  types.StructField('request_datetime', types.TimestampNTZType(), True),
  types.StructField('on_scene_datetime', types.TimestampNTZType(), True),
  types.StructField('pickup_datetime', types.TimestampNTZType(), True),
  types.StructField('dropoff_datetime', types.TimestampNTZType(), True),
  types.StructField('PULocationID', types.IntegerType(), True),
  types.StructField('DOLocationID', types.IntegerType(), True),
  types.StructField('trip_miles', types.DoubleType(), True),
  types.StructField('trip_time', types.IntegerType(), True),
  types.StructField('base_passenger_fare', types.DoubleType(), True),
  types.StructField('tolls', types.DoubleType(), True),
  types.StructField('bcf', types.DoubleType(), True),
  types.StructField('sales_tax', types.DoubleType(), True),
  types.StructField('congestion_surcharge', types.DoubleType(), True),
  types.StructField('airport_fee', types.DoubleType(), True),
  types.StructField('tips', types.DoubleType(), True),
  types.StructField('driver_pay', types.DoubleType(), True),
  types.StructField('shared_request_flag', types.StringType(), True),
  types.StructField('shared_match_flag', types.StringType(), True),
  types.StructField('access_a_ride_flag', types.StringType(), True),
  types.StructField('wav_request_flag', types.StringType(), True),
  types.StructField('wav_match_flag', types.StringType(), True)])

In [27]:
df = spark.read\
      .option("header", True)\
      .parquet('fhvhv_tripdata_2021-01.parquet')
      # .schema(schema)\
      # .option('mergeSchema', True)\

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/bastienwinant/Desktop/Projects/data-engineering-zoomcamp/05-batch-processing/fhvhv_tripdata_2021-01.parquet.

In [ ]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_f

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01', mode='overwrite')

25/02/25 23:16:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/25 23:17:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/02/25 23:17:02 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [ ]:
df\
  .withColumn('pickup_date', F.to_date(df.pickup_date))\
  .withColumn('dropoff_date', F.to_date(df.dropoff_date))\
  .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')\
  .show()

In [ ]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'LOLocationID')\
    .filter(df.hvfhs_license_num == 'HV0003')

In [21]:
spark.stop()